# Task 1

Please be careful about the comment lines!

In [ ]:
#load modules

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [ ]:
#load job descriptions

df_desc = pd.read_csv("data/indeed_us_job_description.csv") # This part need adjustment

In [ ]:
# load job links

df_links = pd.read_csv("data/indeed_us_job_link_withdup.csv") # This part need adjustment

## Explore the Data

### Description Data

---
#### Explore The Data

In [ ]:
df_desc.head()

In [ ]:
df_desc.info()

In [ ]:
len(df_desc.scrape_time.unique()) # I am not sure about the value of scrape_time

In [ ]:
df_desc.job_label.isnull().unique() # whole column is null delete it

In [ ]:
df_desc.rating.isnull().unique() # whole column is null delete it

In [ ]:
len(df_desc.jk.unique()) # jk is the unique key

In [ ]:
len(df_desc.job_page_url.unique()) # job_page_url is unique

In [ ]:
df_desc.job_title.isnull().sum()

In [ ]:
df_desc.company.isnull().sum()

#### Run Cells Below

In [ ]:
df_desc = df_desc[~df_desc.job_title.isnull()]

# use the columns where job_title not null since we will use this column to group the data

In [ ]:
df_desc = df_desc[~df_desc.company.isnull()]

# use the columns where company not null since we will use this column to group the data

#### Do Not Run This Cell Unless You change the indexes of .iat method in the HTML Cleaning Part

In [ ]:
# These columns does not have a value in the calculations

del df_desc["job_label"]
del df_desc["rating"]
del df_desc["state"] # can be reach from "location"
del df_desc["zipcode"]

---

### Links Data

---
#### Explore The Data

In [ ]:
df_links.head()

In [ ]:
# df_links.job_page_url[0]
df_links.job_page_url[1]

# radius stands for how close is the job to you (in miles)

In [ ]:
df_links.search_page_url[0]

In [ ]:
df_links.info()

In [ ]:
df_links[df_links["job_page_url"].isnull()] # no null value

In [ ]:
df_links[df_links["jk"].isnull()] # no null value

In [ ]:
len(df_links["jk"].unique()) 

In [ ]:
len(df_links["job_page_url"].unique()) # same length means jk is created from the job_page_url

# I think this is unnecessary column

In [ ]:
len(df_links.search_count.isnull().unique()) # whole columns is "null" delete it

In [ ]:
df_links[df_links["jk"] == "6516a3814e551ad1"] 

# This matches with your note on Project file " Note that the job link file contains multiple queries 
# from different zip codes for the same job key."

In [ ]:
len(df_links.scrape_time)

---
#### Do Not Run This Cell Unless You change the indexes of .iat method in the HTML Cleaning Part

In [ ]:
# These columns does not have a value in the calculations

del df_links["radius"]
del df_links['search_count']
del df_links["zipcode"]

### 1.1 Merge Data

In [ ]:
# RUN THIS CELL
df_total = pd.merge(df_links, df_desc, on=["jk", "job_page_url"], suffixes=['_links', '_desc'], how="inner" )

# jk and job_page_url are the unique entries for data
# outer not to lose any data I will eliminate the duplicates later

In [ ]:
df_total.info()

# Be avare of the _x and _y columns

In [ ]:
df_total.head(1)

In [ ]:
len(df_total.job_description.isnull().unique()) # no empty job description

In [ ]:
df_total.job_title.isnull().sum() # cleaned the null job_titles

In [ ]:
df_total.company.isnull().sum() # cleaned the null company

In [ ]:
df_total.job_description.isnull().sum() # no None value for the description column

### 1.2 Clean HTML

In [ ]:
text = df_desc.job_description[0]

# Copy paste the list to an Editor to see a structural pattern in this column.

In [ ]:
# Slow code vol.2
# Alternative code block is below

def clean_Job_Description(text):

    clean_text = BeautifulSoup(text, "html.parser").get_text()
    return re.sub(r'\s+', '', clean_text)


for i in range(len(df_total["location"])):
    df_total.iloc[i,16] = clean_Job_Description(df_total.iloc[i,16])

In [ ]:
df_total.job_description[0]

In [ ]:
# Fast code for cleaning the job description

def clean_HTML(text):
    clean_text = re.sub('<[^<]+?>', '', text)  # Remove HTML tags
    clean_text = re.sub('\n', ' ', clean_text)  # Replace \n with a space
    clean_text = re.sub('\s+', ' ', clean_text)  # Replace multiple whitespaces with a single space
    clean_text = re.sub('-', '', clean_text)  # Replace "-" with a space

    return clean_text

for i in range(len(df_total["job_description"])):
    df_total.iat[i,12] = clean_HTML(df_total.iat[i,12])

In [ ]:
df_total.job_description[0]

In [ ]:
# no need to run this cell unless you want to clean the "location" column, too.

for i in range(len(df_total["location"])):
    try:
        df_total.iat[i,16] = clean_HTML(df_total.iat[i,16])
    except TypeError:
        continue

In [ ]:
df_total.location[0]

### 1.3 Remove Duplicate

In [ ]:
# Example to be adjusted

df = pd.DataFrame({
    'jk': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'value': ['A', 'B', 'C', 'B', 'D', 'E', 'A', 'F', 'C', 'G'],
    'company': ['X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X']
})

# Sort the DataFrame by 'company' and 'id'
df_sorted = df.sort_values(['company', 'jk'])

# Find duplicate values based on 'company'
duplicates = df_sorted[df_sorted.duplicated(subset='company', keep=False)]

# Group the duplicates by their company, and create a list of their IDs
duplicates_by_company = duplicates.groupby('company')['jk'].apply(list)

# Drop the duplicates and add a new column for the deleted IDs
df_cleaned = df_sorted.drop_duplicates(subset='company', keep='first').merge(
    duplicates_by_company.rename('jk_list').reset_index(),
    on='company', how='left')

# Fill missing values in the new column with an empty list
df_cleaned['jk_list'] = df_cleaned['jk_list'].fillna(value={})

print(df_cleaned)

---

Codes below works a little slow!
#### Run Cels Below

In [ ]:
df = df_total

In [ ]:
duplicates = df[df_total.duplicated(subset=['company','job_title','job_description'], keep=False)] # works slow

In [ ]:
duplicates.count()

In [ ]:
duplicates_by_company = duplicates.groupby(['company','job_title','job_description'])['jk'].apply(list) 

In [ ]:
df_final = df.drop_duplicates(subset=['company','job_title','job_description']).merge(
    duplicates_by_company.rename('jk_list').reset_index(),
    on=['company','job_title','job_description'], how='inner')

In [ ]:
df_final[df_final.jk_list.isnull()].info()

# I would not expect Nan value in jk_list column since at least rows' own job key should be in the list.
# I am in need of your feedback!

In [ ]:
for i in range(len(df_final["jk_list"])):
    try:
        df_final.iat[i,24] = list(set(df_final.iat[i,24]))
    except TypeError:
        continue

---

### 1.4 Indexing

In [ ]:
df_final['index_dup'] = np.arange(1, df_final.shape[0] + 1) # fast

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv("task1.csv")

---

## Conceptual Questions

Imagine that you receive daily data from 50,000 different job posting / employer sites. The data are in csv, json and other formats with different columns/ fields with approximate 20,000,000 daily job postings in volume.

1) What type of database and platform would you use to process and store the data?

**Answer**: A distributed database system will work for this much of data. It is more safe to use DDS.

2) What technologies and tools would you use?

**Answer:** I have no experience in DB technologies, yet we definitely need the tools that can handle large-scale datasets of different formats.

3) How do you serve it to an end user (via some sort of dashboard/BI tool? Or would you develop something from scratch?

**Answer:** I have used Tableau for learning purposes. It is designed for visualisation of large datasets and user-freiendly. Still, I believe if the code is yours you can achieve more. So, I would create a website to handle the data and visualize it using JS (React or Vue would be enough).